In [1]:
import pandas as pd
df = pd.read_csv('compustat.csv')
df.head()

/var/folders/kp/rt42wzwx7sz9plr9l175fkv40000gn/T/ipykernel_25287/313954475.py:2: DtypeWarning: Columns (10,12,13,17,18,20,25,26,30,32,33,34,35,647,648,649,650,654,656,664,668,669,674,675,678,679) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('compustat.csv')


,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1004,2000-02-29,1999,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
1,1004,2000-05-31,1999,4.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
2,1004,2000-08-31,2000,1.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
3,1004,2000-11-30,2000,2.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
4,1004,2001-02-28,2000,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01


In [2]:
df_with_ratios = pd.DataFrame(df[['datadate', 'gvkey', 'fqtr', 'fyr', 'tic', 'sic']])
df_with_ratios.isna().sum()

datadate      0
gvkey         0
fqtr        346
fyr           0
tic         694
sic           0
dtype: int64

Looks like gvkey would be a better primary key than ticker since it has no nans in this dataset. We also need to inspect financial quarter

In [3]:
df_with_ratios[df['fqtr'].isna()]

,datadate,gvkey,fqtr,fyr,tic,sic
2746,2014-03-31,1224,NaN,12,EGN1,4924
2747,2014-06-30,1224,NaN,12,EGN1,4924
12300,2014-11-30,1951,NaN,8,PRSNQ,3845
31199,2014-12-31,3424,NaN,9,BH,5812
35114,2014-12-31,3806,NaN,9,DWSN,1382
...,...,...,...,...,...,...
1050126,2013-06-30,196766,NaN,3,ALGDF,1040
1050128,2013-09-30,196766,NaN,3,ALGDF,1040
1074260,2015-06-30,317427,NaN,3,NOMD,2030
1074262,2015-09-30,317427,NaN,3,NOMD,2030


No clear pattern in financial quarters with nan values. Just use date value since there is no nans there

In [4]:
df_with_ratios.drop(['fqtr', 'fyr'], axis=1, inplace=True)
df_with_ratios.head()

,datadate,gvkey,tic,sic
0,2000-02-29,1004,AIR,5080
1,2000-05-31,1004,AIR,5080
2,2000-08-31,1004,AIR,5080
3,2000-11-30,1004,AIR,5080
4,2001-02-28,1004,AIR,5080


In [5]:
# Feature engineering
df_with_ratios['ta'] = df['atq']
df_with_ratios['lt'] = df['ltq']
df_with_ratios['dlc'] = df['dlcq']
df_with_ratios['dtp'] = df['txditcq']
df_with_ratios['che'] = df['cheq']
df_with_ratios['lct'] = df['lctq']
df_with_ratios['act'] = df['atq']
df_with_ratios['invtq'] = df['invtq']
df_with_ratios['intanq'] = df['intanq']
df_with_ratios['oiadp'] = df['oibdpq'] / df['oibdpy']
df_with_ratios['txp'] = df['txpq']
# df_with_ratios['xint'] = df['tieq'] / df['tiey']

In [6]:
df_with_ratios['pi'] = df['piq'] / df['piy']
df_with_ratios['ni'] = df['niq'] / df['niy']
df_with_ratios['rev'] = df['revtq'] / df['revty']
df_with_ratios['cogs'] = df['cogsq'] / df['cogsy']
df_with_ratios['oancf'] = df['oancfy']
df_with_ratios['ivncf'] = df['ivncfy']
df_with_ratios['fncfa'] = df['fincfy']
df_with_ratios['curr_ratio'] = df['actq'] / df['lctq']
df_with_ratios['quick_ratio'] = (df['actq'] - df['invtq']) / df['lctq']
df_with_ratios['cash_ratio'] = df['chechy'] / df['lctq']

In [7]:
df_with_ratios['net_working_capital'] = df['actq'] - df['lctq']
df_with_ratios['debt_ratio'] = df['dlttq'] / df['actq']
df_with_ratios['debt_equity_ratio'] = df['dlttq'] / df['ceqq']
df_with_ratios['equity_ratio'] = df['ceqq'] / df['actq']
df_with_ratios['financial_leverage_ratio'] = df['actq'] / df['ceqq']
# df_with_ratios['interest_coverage_ratio'] = (df['uopiq'] + df['dpq']) / df['tiey']
df_with_ratios['cashflow_debt_ratio'] = df['oancfy'] / df['dlttq']
df_with_ratios['net_profit_margin'] = 100 * (df['revtq'] - df['cogsq']) / df['revtq']
df_with_ratios['roa'] = (df['niq'] / df['atq']) * 100

In [8]:
df_with_ratios['asset_turnover'] = df['saleq'] / df['saley']
df_with_ratios['inventory_turnover'] = df['invchy']
df_with_ratios['inventory_days'] = df_with_ratios['inventory_turnover']
df_with_ratios['receivables_turnover'] = df['saleq'] / df['rectq']
df_with_ratios['dso'] = 365 / df_with_ratios['receivables_turnover']
df_with_ratios['working_capital_turnover'] = df['saleq'] / df['wcapq']
# df_with_ratios['debt_service_coverage_ratio'] = df['uopiq'] / df['dlttq']
# df_with_ratios['cash_coverage_ratio'] = df['oibdpq'] / df['tieq']
df_with_ratios['pe_ratio'] = df['prccq'] / df['epsf12']
# df_with_ratios['pb_ratio'] = df['prccq'] / (df['uceqq'] / df['cshoq'])
# df_with_ratios['div_payout_ratio'] = df['dvy'] / ((df['uniamiq'] * 100) / df['saley'])
df_with_ratios['retention_ratio'] = df['req'] / df_with_ratios['ni']
# df_with_ratios['ebitda'] = (df['uopiq'] + df['dpq'] * 100) / df['revtq']

In [9]:
df_with_ratios.head(5)

,datadate,gvkey,tic,sic,ta,lt,dlc,dtp,che,lct,...,net_profit_margin,roa,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,dso,working_capital_turnover,pe_ratio,retention_ratio
0,2000-02-29,1004,AIR,5080,753.755,411.273,25.353,52.701,4.431,176.733,...,18.234795,1.453390,0.340731,-15.536,-15.536,1.606568,227.192332,0.777640,14.751553,603.096525
1,2000-05-31,1004,AIR,5080,740.998,401.483,26.314,56.020,1.241,163.816,...,18.944015,0.333469,0.219732,-33.237,-33.237,1.753662,208.135899,0.647801,10.839844,2860.654487
2,2000-08-31,1004,AIR,5080,747.543,408.290,48.151,58.441,0.669,168.282,...,16.176118,0.422584,1.000000,-8.260,-8.260,1.915678,190.533110,0.706938,11.138614,200.908000
3,2000-11-30,1004,AIR,5080,772.941,431.677,50.437,60.937,3.069,189.367,...,18.857738,0.553470,0.466415,15.085,15.085,1.656685,220.319516,0.649498,13.474026,351.312198
4,2001-02-28,1004,AIR,5080,754.718,409.853,49.665,60.695,0.767,167.852,...,20.259808,0.713909,0.306305,-13.415,-13.415,1.506264,242.321426,0.557790,23.859649,490.594335


In [10]:
print(f"length of dataset: {len(df_with_ratios)}")
print(f"shape of dataset: {df_with_ratios.shape}")

length of dataset: 1075571
shape of dataset: (1075571, 41)


In [11]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.064524
sic,0.000000
ta,24.081441
lt,24.209559
dlc,28.118739
dtp,34.326511
che,24.327729
lct,36.964087


In [12]:

# df_with_ratios['debt_service_coverage_ratio'] = df['uopiq'] / df['dlttq']
# df_with_ratios['xint'] = df['tieq'] / df['tiey']
# df_with_ratios['interest_coverage_ratio'] = (df['uopiq'] + df['dpq']) / df['tiey']
# df_with_ratios['cash_coverage_ratio'] = df['oibdpq'] / df['tieq']
# df_with_ratios['pb_ratio'] = df['prccq'] / (df['uceqq'] / df['cshoq'])
# df_with_ratios['div_payout_ratio'] = df['dvy'] / ((df['uniamiq'] * 100) / df['saley'])
# df_with_ratios['ebitda'] = (df['uopiq'] + df['dpq'] * 100) / df['revtq']

In [13]:
# Run this code to check the percentage of nan valus in the original compustat dataset for a column
# (len(df[df['dlttq'].isna() == True]) / len(df)) * 100

At this point our dataset should have columns with at most 20-30% nan values. we shall now inpute them using sector median. 
Let us first try Methodology 1: 
1. map gvkey to sector
2. Eg. if company X in sector A on date 2000-01-01 has a missing value for total income, we inpute it using the median of sector A in 2000-01-01. 

This assumes that on a particular date, a companies fundamentals can be represented by its sector median on that date. 
Sector will be represented by SIC code. (I used all 4 digits for now)

In [14]:
df_with_ratios.head()

,datadate,gvkey,tic,sic,ta,lt,dlc,dtp,che,lct,...,net_profit_margin,roa,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,dso,working_capital_turnover,pe_ratio,retention_ratio
0,2000-02-29,1004,AIR,5080,753.755,411.273,25.353,52.701,4.431,176.733,...,18.234795,1.453390,0.340731,-15.536,-15.536,1.606568,227.192332,0.777640,14.751553,603.096525
1,2000-05-31,1004,AIR,5080,740.998,401.483,26.314,56.020,1.241,163.816,...,18.944015,0.333469,0.219732,-33.237,-33.237,1.753662,208.135899,0.647801,10.839844,2860.654487
2,2000-08-31,1004,AIR,5080,747.543,408.290,48.151,58.441,0.669,168.282,...,16.176118,0.422584,1.000000,-8.260,-8.260,1.915678,190.533110,0.706938,11.138614,200.908000
3,2000-11-30,1004,AIR,5080,772.941,431.677,50.437,60.937,3.069,189.367,...,18.857738,0.553470,0.466415,15.085,15.085,1.656685,220.319516,0.649498,13.474026,351.312198
4,2001-02-28,1004,AIR,5080,754.718,409.853,49.665,60.695,0.767,167.852,...,20.259808,0.713909,0.306305,-13.415,-13.415,1.506264,242.321426,0.557790,23.859649,490.594335


First let us convert datadate to year and quarter representation

In [15]:
df_with_ratios['datadate'] = pd.to_datetime(df_with_ratios['datadate'])
df_with_ratios['year'] = df_with_ratios['datadate'].dt.year
df_with_ratios['quarter'] = df_with_ratios['datadate'].dt.quarter
df_with_ratios.head()

,datadate,gvkey,tic,sic,ta,lt,dlc,dtp,che,lct,...,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,dso,working_capital_turnover,pe_ratio,retention_ratio,year,quarter
0,2000-02-29,1004,AIR,5080,753.755,411.273,25.353,52.701,4.431,176.733,...,0.340731,-15.536,-15.536,1.606568,227.192332,0.777640,14.751553,603.096525,2000,1
1,2000-05-31,1004,AIR,5080,740.998,401.483,26.314,56.020,1.241,163.816,...,0.219732,-33.237,-33.237,1.753662,208.135899,0.647801,10.839844,2860.654487,2000,2
2,2000-08-31,1004,AIR,5080,747.543,408.290,48.151,58.441,0.669,168.282,...,1.000000,-8.260,-8.260,1.915678,190.533110,0.706938,11.138614,200.908000,2000,3
3,2000-11-30,1004,AIR,5080,772.941,431.677,50.437,60.937,3.069,189.367,...,0.466415,15.085,15.085,1.656685,220.319516,0.649498,13.474026,351.312198,2000,4
4,2001-02-28,1004,AIR,5080,754.718,409.853,49.665,60.695,0.767,167.852,...,0.306305,-13.415,-13.415,1.506264,242.321426,0.557790,23.859649,490.594335,2001,1


In [16]:
df_drop_duplicates = df_with_ratios.drop_duplicates(subset=['gvkey', 'year', 'quarter'])
len(df_with_ratios) - len(df_drop_duplicates)

1402

We then drop rows where there are duplicate year, quarters and gvkey, since the original dataset is monthly data. We only drop 1402 rows, so not significant. 

In [17]:
df_with_ratios = df_drop_duplicates

Now we find all the gvkeys where there is no data at all and drop them from our dataframe. From this operation we remove 6813 gvkeys from a total of 

In [18]:
df_drop_gvkey = df_with_ratios.drop(['sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
df_drop_gvkey = df_drop_gvkey.groupby('gvkey').apply(lambda group: group.iloc[:, 1:].isna().all().all())
to_drop = df_drop_gvkey[df_drop_gvkey == True]
gvkeys_to_drop = list(to_drop.index)

In [19]:
original_number_of_gvkeys = len(df_with_ratios['gvkey'].unique())
print(f"Dropped {len(gvkeys_to_drop) * 100 / original_number_of_gvkeys}% of stocks")

Dropped 22.53381394887397% of stocks


In [20]:
df_with_ratios_dropped_gvkey = df_with_ratios[~df_with_ratios['gvkey'].isin(gvkeys_to_drop)]
print(f"number of unique gvkeys now: {len(df_with_ratios_dropped_gvkey['gvkey'].unique())}")

number of unique gvkeys now: 23425


In [21]:
print(f"% of total rows lost: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey)) * 100 / len(df_with_ratios)  }%")

% of total rows lost: 17.750651899282143%


In [22]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey.isna().sum() / len(df_with_ratios_dropped_gvkey)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.027957
sic,0.000000
ta,7.687519
lt,7.828550
dlc,12.597439
dtp,20.155247
che,7.986445
lct,23.362954


We can also see that the % of nans has decreased significantly across columns after dropping these gvkeys

In [23]:
test = df_with_ratios_dropped_gvkey
test = test[test.isna().any(axis=1)]
test[test['tic'] == 'AFAP']

,datadate,gvkey,tic,sic,ta,lt,dlc,dtp,che,lct,...,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,dso,working_capital_turnover,pe_ratio,retention_ratio,year,quarter
156,2000-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,24.154589,NaN,2000,1
157,2000-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,0.509702,NaN,NaN,NaN,NaN,NaN,21.290323,NaN,2000,2
158,2000-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,3
159,2000-12-31,1019,AFAP,7380,28.638,15.454,0.666,0.543,1.818,5.390,...,0.490777,NaN,NaN,3.228103,113.069511,6.046377,17.395437,24.544079,2000,4
160,2001-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,16.726297,NaN,2001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-12-31,1019,AFAP,7380,33.355,21.722,1.833,0.532,1.865,16.881,...,1.000000,-0.612,-0.612,5.154342,70.814077,12.136722,13.225569,8.416000,2019,4
236,2020-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1
237,2020-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2
238,2020-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,3


Here we can see that there are still certain stocks where there is data, but many missing values. for example for ticker AFAP, we have data missing from 2003-2016. Such tickers should be dropped as well

In [24]:
cols_df = df_with_ratios_dropped_gvkey.drop(['gvkey', 'sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
all_na_rows = cols_df[cols_df.isna().all(axis=1)]
df_with_nans = df_with_ratios_dropped_gvkey.loc[all_na_rows.index]
df_with_nans_counts = df_with_nans.groupby('gvkey').size()
gvkey_with_bad_data = df_with_nans_counts[df_with_nans_counts > 10]

In [25]:
print(len(df_with_ratios_dropped_gvkey['gvkey'].unique()))
print(100*len(gvkey_with_bad_data) / len(df_with_ratios_dropped_gvkey['gvkey'].unique()))

23425
2.864461045891142


We can decide here the threshold for which we want to drop our gvkeys. How many nan rows can we allow a gvkey to have? For now i set the value at less than 10. This causes us to drop 671 gvkeys, or 2.86% of gvkeys. 

In [26]:
df_with_ratios_dropped_gvkey_v2 = df_with_ratios_dropped_gvkey[~df_with_ratios_dropped_gvkey['gvkey'].isin(gvkey_with_bad_data.index)]
len(df_with_ratios_dropped_gvkey_v2)

844831

In [27]:
print(f"% of total rows lost: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey_v2)) * 100 / len(df_with_ratios)  }%")

% of total rows lost: 21.350271698401276%


In [28]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.022995
sic,0.000000
ta,4.754466
lt,4.860222
dlc,8.674613
dtp,14.755406
che,4.981339
lct,17.225129


In [29]:
nan_breakdown['pct_nan'].mean()

10.00147718300011

Our % of rows dropped increased from 17% to 21%. We can reduce this perentage by increasing the tolerance of how many nan rows a stock can have. This is balancing data quantity vs quality. 

In [30]:
sector_date_median = df_with_ratios_dropped_gvkey_v2.groupby(['sic', 'year', 'quarter']).median()
sector_date_median

gvkey        ta       lt      dlc     dtp      che  \
sic  year quarter                                                         
100  2000 1        26100.5  135.9875  63.1280   3.1075  0.0000   3.8000   
          2        26100.5  170.3230  70.1710   2.5400  0.0000   6.2990   
          3        24900.0  183.2935  91.3005  13.8835  0.0000   6.5265   
          4        25190.5  261.4290  99.8750  22.8430  0.5995  10.6000   
     2001 1        27301.0  188.9755  64.1785   5.0435  0.0000   7.5840   
...                    ...       ...      ...      ...     ...      ...   
9998 2010 4        22707.0   23.6580   0.3280   0.0000  0.0000   4.4090   
     2011 1        22707.0   23.5840   0.3310   0.0000  0.0000   5.2070   
          2        22707.0   24.8190   0.3450   0.0000  0.0000   5.3220   
          3        22707.0   25.9010   0.3210   0.0000  0.0000   5.5430   
          4        22707.0   26.2730   0.2900   0.0000  0.0000   5.8310   

                       lct       act    invtq  intanq  ...  net_profit_margin  \
sic  year quarter                                      ...                      
100  2000 1        25.2875  135.9875  13.0340     NaN  ...          27.073591   
          2        23.3750  170.3230  16.6470     NaN  ...          31.971606   
          3        58.4605  183.2935  24.2520     NaN  ...          34.283935   
          4        60.8015  261.4290  48.3020     NaN  ...          29.187245   
     2001 1        20.8235  188.9755  18.4565     NaN  ...          31.844447   
...                    ...       ...      ...     ...  ...                ...   
9998 2010 4         0.3280   23.6580   0.0000     0.0  ...                NaN   
     2011 1         0.3310   23.5840   0.0000     0.0  ...                NaN   
          2         0.3450   24.8190   0.0000     0.0  ...                NaN   
          3         0.3210   25.9010   0.0000     0.0  ...                NaN   
          4         0.2900   26.2730   0.0000     0.0  ...                NaN   

                        roa  asset_turnover  inventory_turnover  \
sic  year quarter                                                 
100  2000 1        0.387070        1.000000             -1.4150   
          2        0.306799        0.490545              0.0480   
          3       -0.895477        0.268579              0.0265   
          4       -0.695809        0.368586             -0.5985   
     2001 1        0.459175        1.000000             -0.1840   
...                     ...             ...                 ...   
9998 2010 4        1.327247             NaN              0.0000   
     2011 1       -0.326493             NaN              0.0000   
          2        4.919618             NaN              0.0000   
          3        4.270105             NaN              0.0000   
          4        1.533894             NaN              0.0000   

                   inventory_days  receivables_turnover         dso  \
sic  year quarter                                                     
100  2000 1               -1.4150              1.256291  290.537691   
          2                0.0480              1.865352  195.674495   
          3                0.0265              1.167319  312.682355   
          4               -0.5985              1.708163  213.679809   
     2001 1               -0.1840              1.414857  257.976672   
...                           ...                   ...         ...   
9998 2010 4                0.0000                   NaN         NaN   
     2011 1                0.0000                   NaN         NaN   
          2                0.0000                   NaN         NaN   
          3                0.0000                   NaN         NaN   
          4                0.0000                   NaN         NaN   

                   working_capital_turnover  pe_ratio  retention_ratio  
sic  year quarter                                                       
100  2000 1                        0.715197  2.62

In [31]:
sector_date_median[sector_date_median.index == (100,2000,1)]['che'].iloc[0]

3.8

In [32]:
cols_to_check = [col for col in df_with_ratios_dropped_gvkey_v2.columns if col not in ['datadate', 'year', 'quarter', 'sic', 'gvkey', 'tic']]
cols_to_check

['ta',
 'lt',
 'dlc',
 'dtp',
 'che',
 'lct',
 'act',
 'invtq',
 'intanq',
 'oiadp',
 'txp',
 'pi',
 'ni',
 'rev',
 'cogs',
 'oancf',
 'ivncf',
 'fncfa',
 'curr_ratio',
 'quick_ratio',
 'cash_ratio',
 'net_working_capital',
 'debt_ratio',
 'debt_equity_ratio',
 'equity_ratio',
 'financial_leverage_ratio',
 'cashflow_debt_ratio',
 'net_profit_margin',
 'roa',
 'asset_turnover',
 'inventory_turnover',
 'inventory_days',
 'receivables_turnover',
 'dso',
 'working_capital_turnover',
 'pe_ratio',
 'retention_ratio']

In [34]:
# %%time
import numpy as np
def fill_with_median(x):
    sic = x['sic']
    year = x['year']
    quarter = x['quarter']
    for col in cols_to_check: 
        if np.isnan(x[col]): 
            median = sector_date_median[sector_date_median.index == (sic,year,quarter)][col].iloc[0]
            x[col] = median
    return x

cols_median = ['sic', 'year', 'quarter']

df_inputed1 = df_with_ratios_dropped_gvkey_v2[:50000].apply(fill_with_median, axis=1)

In [36]:
df_inputed2 = df_with_ratios_dropped_gvkey_v2[50000:100000].apply(fill_with_median, axis=1)
df_inputed3 = df_with_ratios_dropped_gvkey_v2[100000:150000].apply(fill_with_median, axis=1)
df_inputed4 = df_with_ratios_dropped_gvkey_v2[150000:200000].apply(fill_with_median, axis=1)


In [37]:
df_inputed5 = df_with_ratios_dropped_gvkey_v2[200000:250000].apply(fill_with_median, axis=1)
df_inputed6 = df_with_ratios_dropped_gvkey_v2[250000:300000].apply(fill_with_median, axis=1)

In [46]:
df_inputed7 = df_with_ratios_dropped_gvkey_v2[300000:350000].apply(fill_with_median, axis=1)
df_inputed8 = df_with_ratios_dropped_gvkey_v2[350000:400000].apply(fill_with_median, axis=1)
df_inputed9 = df_with_ratios_dropped_gvkey_v2[400000:450000].apply(fill_with_median, axis=1)

In [48]:
combined = pd.DataFrame()
for df in [
    df_inputed1,
    df_inputed2,
        df_inputed3,
    df_inputed4,
        df_inputed5,
    df_inputed6,
        df_inputed7,
    df_inputed8,
    df_inputed9,
]:
    combined = pd.concat([combined, df])
    
    

In [53]:
df_inputed10 = df_with_ratios_dropped_gvkey_v2[450000:500000].apply(fill_with_median, axis=1)
df_inputed11 = df_with_ratios_dropped_gvkey_v2[500000:550000].apply(fill_with_median, axis=1)
df_inputed12 = df_with_ratios_dropped_gvkey_v2[550000:600000].apply(fill_with_median, axis=1)

In [54]:
df_inputed13 = df_with_ratios_dropped_gvkey_v2[600000:650000].apply(fill_with_median, axis=1)
df_inputed14 = df_with_ratios_dropped_gvkey_v2[650000:700000].apply(fill_with_median, axis=1)
df_inputed15 = df_with_ratios_dropped_gvkey_v2[700000:750000].apply(fill_with_median, axis=1)

In [55]:

df_inputed16 = df_with_ratios_dropped_gvkey_v2[750000:800000].apply(fill_with_median, axis=1)
df_inputed17 = df_with_ratios_dropped_gvkey_v2[800000:850000].apply(fill_with_median, axis=1)
df_inputed18 = df_with_ratios_dropped_gvkey_v2[850000:].apply(fill_with_median, axis=1)

In [56]:
for df in [
    df_inputed10,
    df_inputed11,
        df_inputed12,
    df_inputed13,
        df_inputed14,
    df_inputed15,
        df_inputed16,
    df_inputed17,
    df_inputed18,
]:
    combined = pd.concat([combined, df])

In [57]:
combined.to_pickle('./combined_final.pkl')

In [61]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan_before_sector_inpute'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute'] = (combined.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan_before_sector_inpute,pct_nan_after_sector_inpute
datadate,0.000000,0.000000
gvkey,0.000000,0.000000
tic,0.022995,0.022995
sic,0.000000,0.000000
ta,4.754466,0.018340
lt,4.860222,0.018712
dlc,8.674613,0.036866
dtp,14.755406,0.457098
che,4.981339,0.018433
lct,17.225129,5.692680


In [63]:
inputed_data_ffill = combined.ffill(limit=4)

In [66]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan_before_sector_inpute'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute'] = (combined.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute_and_ffill'] = (inputed_data_ffill.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan_before_sector_inpute,pct_nan_after_sector_inpute,pct_nan_after_sector_inpute_and_ffill
datadate,0.000000,0.000000,0.000000
gvkey,0.000000,0.000000,0.000000
tic,0.022995,0.022995,0.020388
sic,0.000000,0.000000,0.000000
ta,4.754466,0.018340,0.000000
lt,4.860222,0.018712,0.000652
dlc,8.674613,0.036866,0.001490
dtp,14.755406,0.457098,0.282823
che,4.981339,0.018433,0.000000
lct,17.225129,5.692680,4.342054


In [67]:
print(len(df_with_ratios))
print(len(inputed_data_ffill))

1074169
844831


In [68]:
inputed_data_ffill_dropna = inputed_data_ffill.dropna()

In [69]:
len(inputed_data_ffill_dropna)

787972

In [70]:
print(len(df_with_ratios['gvkey'].unique()))
print(len(inputed_data_ffill_dropna['gvkey'].unique()))

30239
22672


In [71]:
inputed_data_ffill_dropna.to_pickle('./compustat_inputed.pkl')

In [72]:
inputed_data_ffill_dropna

,datadate,gvkey,tic,sic,ta,lt,dlc,dtp,che,lct,...,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,dso,working_capital_turnover,pe_ratio,retention_ratio,year,quarter
7,2001-11-30,1004,AIR,5080,714.2080,429.8640,71.162,55.1340,53.129,183.797,...,0.416489,-8.6610,-8.6610,1.828483,199.619019,0.502881,-5.093750,146.623341,2001,4
8,2002-02-28,1004,AIR,5080,690.6810,375.8490,20.890,55.3140,34.454,129.670,...,0.291972,-16.7510,-16.7510,2.005354,182.012763,0.447152,-3.957219,3546.832238,2002,1
9,2002-05-31,1004,AIR,5080,710.1990,399.9640,42.525,30.6010,34.522,150.464,...,0.230746,-19.5200,-19.5200,1.901016,192.002551,0.514976,-5.500000,3103.757532,2002,2
10,2002-08-31,1004,AIR,5080,722.9440,416.3840,41.989,25.7170,35.553,142.645,...,1.000000,12.3600,12.3600,2.096486,174.100883,0.548749,-2.649573,135.885000,2002,3
11,2002-11-30,1004,AIR,5080,727.7760,421.5420,91.772,28.1160,39.310,197.359,...,0.503100,1.7800,1.7800,2.033927,179.455770,0.648304,-16.666667,1132.941949,2002,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075564,2021-12-31,353945,ACLLY,3621,686.0850,379.0350,102.676,16.3000,73.183,326.123,...,1.000000,4.1540,4.1540,3.672771,99.379999,6.831010,8.210526,-47.818582,2021,4
1075565,2022-03-31,353945,ACLLY,3621,494.4620,300.7910,17.676,3.0390,38.581,78.767,...,1.000000,-6.0100,-6.0100,1.449870,251.746676,0.677785,20.625000,22.297000,2022,1
1075566,2022-06-30,353945,ACLLY,3621,407.2310,75.4930,6.954,0.1485,33.225,64.711,...,0.532039,-6.3005,-6.3005,1.578615,231.215355,0.677005,16.630451,5.297990,2022,2
1075567,2022-09-30,353945,ACLLY,3621,488.3815,228.0905,8.396,0.2030,35.454,77.485,...,0.358163,-13.7490,-13.7490,1.490826,246.804261,0.588082,-4.336634,11.068578,2022,3
